In [3]:
import os
import praw
from dotenv import load_dotenv

load_dotenv()

REDDIT_CLIENT_ID = os.environ["REDDIT_CLIENT_ID"]
REDDIT_CLIENT_SECRET = os.environ["REDDIT_CLIENT_SECRET"]
REDDIT_PASSWORD = os.environ["REDDIT_PASSWORD"]
REDDIT_USERNAME = os.environ["REDDIT_USERNAME"]
REDDIT_AGENT_NAME = os.environ["REDDIT_AGENT_NAME"]
SUBREDDIT_NAME = os.environ["SUBREDDIT_NAME"]

In [4]:
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_AGENT_NAME,
    username=REDDIT_USERNAME,
    password=REDDIT_PASSWORD,
)

subreddit = reddit.subreddit(SUBREDDIT_NAME)


def fetch_new_submissions(subreddit: any, limit: int = 10) -> list:
    submissions = []
    for submission in subreddit.new(limit=limit):
        submissions.append(submission)
    return submissions


def fetch_hot_submissions(subreddit: any, limit: int = 10) -> list:
    submissions = []
    for submission in subreddit.hot(limit=limit):
        submissions.append(submission)
    return submissions

In [ ]:
subbmissions = fetch_hot_submissions(subreddit, limit=5)

'https://v.redd.it/up3aaplbmdte1'

In [ ]:
import os
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
 
SCOPES = ['https://www.googleapis.com/auth/youtube.upload']

token_filepath = os.environ["TOKEN_FILEPATH"]
clinet_secrets_filepath = os.environ["CLIENT_SECRETS_FILEPATH"]

def authenticate_youtube():
    creds = None
 
    if os.path.exists(token_filepath):
        creds = Credentials.from_authorized_user_file(token_filepath, SCOPES)
 
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(clinet_secrets_filepath, SCOPES)
            creds = flow.run_local_server(port=0)
 
        with open(token_filepath, 'w') as token:
            token.write(creds.to_json())
    return build('youtube', 'v3', credentials=creds)

def upload_video(youtube, file_path, title, description, tags, category_id, privacy_status):
    body=dict(
        snippet=dict(
            title=title,
            description=description,
            tags=tags,
            categoryId=category_id
        ),
        status=dict(
            privacyStatus=privacy_status
        )
    )
    media = MediaFileUpload(file_path, chunksize=-1, resumable=True)
    request = youtube.videos().insert(
        part=','.join(body.keys()),
        body=body,
        media_body=media
    )
    response = None
    while response is None:
        status, response = request.next_chunk()
        if status:
            print(f"Uploaded {int(status.progress() * 100)}%")
    print(f"Upload complete! Video ID: {response['id']}")

In [ ]:
from redvid import Downloader

path = f"./downloads/{SUBREDDIT_NAME}"

os.makedirs(path, exist_ok=True)

for submission in subbmissions:

    reddit = Downloader(max_q=True,path= path)
    reddit.url = submission.url
    reddit.download()
    
    youtube = authenticate_youtube()
    upload_video(
        youtube,
        file_path=path,
        title=submission.title,
        description=submission.selftext,
        tags=['tag1', 'tag2'],
        category_id='22',   
        privacy_status='private'  
    )